In [1]:
!gdown 19LSrZHYQqJSdKgH8Mtlgg7-i-L3eRhbh

Downloading...
From (original): https://drive.google.com/uc?id=19LSrZHYQqJSdKgH8Mtlgg7-i-L3eRhbh
From (redirected): https://drive.google.com/uc?id=19LSrZHYQqJSdKgH8Mtlgg7-i-L3eRhbh&confirm=t&uuid=0add35d3-4482-4d50-8218-9987e8bca63e
To: /content/D-Fire.zip
100% 3.04G/3.04G [01:11<00:00, 42.6MB/s]


In [2]:
!unzip D-Fire.zip -d datasets/

Streaming output truncated to the last 5000 lines.
  inflating: datasets/train/labels/WEB04443.txt  
  inflating: datasets/train/labels/WEB04444.txt  
  inflating: datasets/train/labels/WEB04445.txt  
  inflating: datasets/train/labels/WEB04446.txt  
  inflating: datasets/train/labels/WEB04447.txt  
  inflating: datasets/train/labels/WEB04448.txt  
  inflating: datasets/train/labels/WEB04449.txt  
  inflating: datasets/train/labels/WEB04450.txt  
  inflating: datasets/train/labels/WEB04451.txt  
  inflating: datasets/train/labels/WEB04452.txt  
  inflating: datasets/train/labels/WEB04453.txt  
  inflating: datasets/train/labels/WEB04454.txt  
  inflating: datasets/train/labels/WEB04455.txt  
  inflating: datasets/train/labels/WEB04456.txt  
  inflating: datasets/train/labels/WEB04457.txt  
  inflating: datasets/train/labels/WEB04458.txt  
  inflating: datasets/train/labels/WEB04459.txt  
  inflating: datasets/train/labels/WEB04460.txt  
  inflating: datasets/train/labels/WEB04461.txt  

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import os
import shutil
import random

# Paths
train_img_dir = "datasets/train/images"
train_lbl_dir = "datasets/train/labels"
val_img_dir = "datasets/val/images"
val_lbl_dir = "datasets/val/labels"

# Create validation directories if they don't exist
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(train_img_dir) if f.endswith('.jpg')]

# Define validation split (e.g., 10%)
val_ratio = 0.1
val_size = int(len(image_files) * val_ratio)

# Select random images for validation
val_images = random.sample(image_files, val_size)

for img in val_images:
    label = img.replace(".jpg", ".txt")  # Get corresponding label filename

    # Move image
    shutil.move(os.path.join(train_img_dir, img), os.path.join(val_img_dir, img))

    # Move label (if it exists)
    if os.path.exists(os.path.join(train_lbl_dir, label)):
        shutil.move(os.path.join(train_lbl_dir, label), os.path.join(val_lbl_dir, label))

print(f"Moved {val_size} images and labels to validation set.")


Moved 1722 images and labels to validation set.


In [5]:
text = """
# data.yaml
train: train/images
val: val/images

# Number of classes (e.g., fire and smoke)
nc: 2

# Class names (ensure the order matches the annotation labels)
names: ['smoke', 'fire']
"""
with open("data.yaml", "w+") as f:
  f.write(text)

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load an official model

PROJECT = 'FireDetection'  # project name
NAME = 'FireExp'  # run name

model.train(
   data = 'data.yaml',
   task = 'detect',
   epochs = 10,
   verbose = True,
   batch = 64,
   imgsz = 640,
   patience = 5,
   save = True,
   device = 0,
   workers = 8,
   project = PROJECT,
   name = NAME,
   cos_lr = True,
   lr0 = 0.0001,
   lrf = 0.00001,
   warmup_epochs = 3,
   warmup_bias_lr = 0.000001,
   optimizer = 'Adam',
   seed = 42,
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 315MB/s]


Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=5, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=FireDetection, name=FireExp, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

100%|██████████| 755k/755k [00:00<00:00, 118MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      10/10      9.62G       1.38      1.472      1.284         13        640: 100%|██████████| 243/243 [04:46<00:00,  1.18s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:20<00:00,  1.50s/it]


                   all       1722       2163      0.739      0.608      0.702      0.404

10 epochs completed in 0.844 hours.
Optimizer stripped from FireDetection/FireExp/weights/last.pt, 6.2MB
Optimizer stripped from FireDetection/FireExp/weights/best.pt, 6.2MB

Validating FireDetection/FireExp/weights/best.pt...
Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:34<00:00,  2.44s/it]


                   all       1722       2163      0.736      0.611      0.702      0.404
                 smoke        866        970      0.797      0.681      0.771      0.472
                  fire        484       1193      0.674      0.541      0.633      0.336
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to FireDetection/FireExp


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b08322695d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [6]:
# # text = """
# # # data.yaml
# # train: train/images
# # val: val/images

# # # Number of classes (e.g., fire and smoke)
# # nc: 2

# # # Class names (ensure the order matches the annotation labels)
# # names: ['smoke', 'fire']
# # """
# # with open("data.yaml", "w+") as f:
# #   f.write(text)

# from ultralytics import YOLO

# # Load a model
# model = YOLO('yolov11n.pt')  # load an official model

# PROJECT = 'FireDetection'  # project name
# NAME = 'FireExpFinal'  # run name

# model.train(
#    data = 'data.yaml',
#    task = 'detect',
#    epochs = 10,
#    verbose = True,
#    batch = 64,
#    imgsz = 640,
#    patience = 5,
#    save = True,
#    device = 0,
#    workers = 8,
#    project = PROJECT,
#    name = NAME,
#    cos_lr = True,
#    lr0 = 0.0001,
#    lrf = 0.00001,
#    warmup_epochs = 3,
#    warmup_bias_lr = 0.000001,
#    optimizer = 'Adam',
#    seed = 42,
# )